# Assignment 3 - PART 1

# Section A - Working with the RDD API

In [1]:
from pyspark.sql import SparkSession
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.87:7077") \
        .appName("Nithesh_A1")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",4)\
        .getOrCreate()
spark_context = spark_session.sparkContext

## Solutions to Question A.1

In [2]:
#A.1.1 Read the English transcripts with Spark, and count the number of lines.
english = spark_context.newAPIHadoopFile(
    'hdfs://192.168.2.87:9000/europarl/europarl-v7.sv-en.en',
    'org.apache.hadoop.mapreduce.lib.input.TextInputFormat',
    'org.apache.hadoop.io.LongWritable',
    'org.apache.hadoop.io.Text',
   # conf={'textinputformat.record.delimiter': 'WARC/1.0'}
)\
.cache() # Keep this RDD in memory!
print('Number of lines in English Transcript:', english.count())

Number of lines in English Transcript: 1862234


In [3]:
# A.1.2 Do the same with the other language (so that you have a separate lineage of RDDs for each).
svenska =  spark_context.newAPIHadoopFile(
    'hdfs://192.168.2.87:9000/europarl/europarl-v7.sv-en.sv',
    'org.apache.hadoop.mapreduce.lib.input.TextInputFormat',
    'org.apache.hadoop.io.LongWritable',
    'org.apache.hadoop.io.Text',
   # conf={'textinputformat.record.delimiter': 'WARC/1.0'}
)\
.cache() # Keep this RDD in memory!
print('Number of lines in Swedish Transcript:', svenska.count())

Number of lines in Swedish Transcript: 1862234


In [4]:
# A.1.3 Verify that the line counts are the same for the two languages.
if english.count() == svenska.count():
    print('The line counts are same for the two languages')
else:
    print('The line counts are not same for the two languages')

The line counts are same for the two languages


In [5]:
# A.1.4 Count the number of partitions.
print('Number of partitions for first RDD is', english.getNumPartitions())
print('Number of partitions for second RDD is', svenska.getNumPartitions())

Number of partitions for first RDD is 2
Number of partitions for second RDD is 3


## Solution to Question A.2 

In [6]:
#A.2.1 Pre-process the text from both RDDs by doing the following:
#● Lowercase the text
#● Tokenize the text (split on space)
def preprocess(rdd):
    return rdd\
.map(lambda word: word[1].lower())\
.map(lambda line: line.split(' '))




In [7]:
# A.2.2 Inspect 10 entries from each of your RDDs to verify your pre-processing.

#For English:
rdd1 = preprocess(english)
rdd1.take(10)

[['resumption', 'of', 'the', 'session'],
 ['i',
  'declare',
  'resumed',
  'the',
  'session',
  'of',
  'the',
  'european',
  'parliament',
  'adjourned',
  'on',
  'friday',
  '17',
  'december',
  '1999,',
  'and',
  'i',
  'would',
  'like',
  'once',
  'again',
  'to',
  'wish',
  'you',
  'a',
  'happy',
  'new',
  'year',
  'in',
  'the',
  'hope',
  'that',
  'you',
  'enjoyed',
  'a',
  'pleasant',
  'festive',
  'period.'],
 ['although,',
  'as',
  'you',
  'will',
  'have',
  'seen,',
  'the',
  'dreaded',
  "'millennium",
  "bug'",
  'failed',
  'to',
  'materialise,',
  'still',
  'the',
  'people',
  'in',
  'a',
  'number',
  'of',
  'countries',
  'suffered',
  'a',
  'series',
  'of',
  'natural',
  'disasters',
  'that',
  'truly',
  'were',
  'dreadful.'],
 ['you',
  'have',
  'requested',
  'a',
  'debate',
  'on',
  'this',
  'subject',
  'in',
  'the',
  'course',
  'of',
  'the',
  'next',
  'few',
  'days,',
  'during',
  'this',
  'part-session.'],
 ['in',
  

In [8]:
#For Swedish:
rdd2 = preprocess(svenska)
rdd2.take(10)


[['återupptagande', 'av', 'sessionen'],
 ['jag',
  'förklarar',
  'europaparlamentets',
  'session',
  'återupptagen',
  'efter',
  'avbrottet',
  'den',
  '17',
  'december.',
  'jag',
  'vill',
  'på',
  'nytt',
  'önska',
  'er',
  'ett',
  'gott',
  'nytt',
  'år',
  'och',
  'jag',
  'hoppas',
  'att',
  'ni',
  'haft',
  'en',
  'trevlig',
  'semester.'],
 ['som',
  'ni',
  'kunnat',
  'konstatera',
  'ägde',
  '"den',
  'stora',
  'år',
  '2000-buggen"',
  'aldrig',
  'rum.',
  'däremot',
  'har',
  'invånarna',
  'i',
  'ett',
  'antal',
  'av',
  'våra',
  'medlemsländer',
  'drabbats',
  'av',
  'naturkatastrofer',
  'som',
  'verkligen',
  'varit',
  'förskräckliga.'],
 ['ni',
  'har',
  'begärt',
  'en',
  'debatt',
  'i',
  'ämnet',
  'under',
  'sammanträdesperiodens',
  'kommande',
  'dagar.'],
 ['till',
  'dess',
  'vill',
  'jag',
  'att',
  'vi,',
  'som',
  'ett',
  'antal',
  'kolleger',
  'begärt,',
  'håller',
  'en',
  'tyst',
  'minut',
  'för',
  'offren',
  'f

In [9]:
# A.2.3 Verify that the line counts still match after the pre-processing.
if rdd1.count() == rdd2.count():
    print('The line counts still match after the pre-processing and the total number of lines is ',rdd1.count())
else:
    print('The line counts does not match. For English transcript, the line count is ',rdd1.count(),' and for swedish transcript, the line count is ',rdd2.count())

The line counts still match after the pre-processing and the total number of lines is  1862234


## Solutions to Question A.3

In [ ]:
# A.3.1 Use Spark to compute the 10 most frequently occuring words in the English language corpus. 
from operator import add

def frequentwords(rdd):
    return rdd\
.flatMap(lambda word: word)\
.map(lambda word: (word,1))

#For English:
rdd3 = frequentwords(rdd1)
freq_en = rdd3.reduceByKey(add)

#For Swedish:
rdd4 = frequentwords(rdd2)
freq_sv = rdd4.reduceByKey(add)

print(freq_en.takeOrdered(10, key=lambda x: -x[1]))
print(freq_sv.takeOrdered(10, key=lambda x: -x[1]))


[('the', 3498375), ('of', 1659758), ('to', 1539760), ('and', 1288401), ('in', 1085993), ('that', 797516), ('a', 773522), ('is', 758050), ('for', 534242), ('we', 522849)]
[('att', 1706293), ('och', 1344830), ('i', 1050774), ('det', 924866), ('som', 913276), ('för', 908680), ('av', 738068), ('är', 694381), ('en', 620310), ('vi', 539797)]


## Solutions to Question A.4

In [ ]:
# A.4.1 Use this parallel corpus to mine some translations in the form of word pairs, for the two languages. 

rdden = rdd1.zipWithIndex()\
.map(lambda x: (x[1],x[0]))\

rddsv = rdd2.zipWithIndex()\
.map(lambda x: (x[1],x[0]))\

rddjoin = rdden.join(rddsv)
rddkeys = rddjoin.reduceByKey(lambda x,y : (x[1], y[1]))\

filter1 = rddkeys.filter(lambda x: x[1][0] != " ")\
                 .filter(lambda x: x[1][1] != " ")\

filter2 = filter1.filter(lambda x: len(x[1][0]) == len(x[1][1]))

en_words = filter2.flatMap(lambda x: x[1][0])
sv_words = filter2.flatMap(lambda x: x[1][1])

en_words.take(5)

joinwords=en_words.zip(sv_words)

mapper=joinwords.map(lambda x: (x,1))

dictionary = mapper.reduceByKey(add)
print(dictionary.takeOrdered(20, key=lambda x: -x[1]))
                   



In [ ]:
# release the cores for another application!
spark_context.stop()